<h1><center>Lending Club EDA</center></h1>

<h2><center>Introduction</center></h2>


Lending Club is the world's largest peer-to-peer lending platform. They offer lending services by connecting borrowers with investors through an online marketplace. While this has become a popular source of passive income for investors, there is also a risk of default, in which the borrowers fall behind on their repayments. In this kernel, we will perform some explorations and feature analysis on the loan data provided by Lending Club to identify the potential risk factors associated with a loan default.

<h2><center>Getting Started</center></h2>

Lets import all the required libraries and take a look at the data

In [ ]:
# Importing all libraries required.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Ignore warnings due to version problems
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Read CSV file and check and load the data set for further analysis 
loan = pd.read_csv('loan.csv')
loan.head()

## Observation from the first 5 rows

1. There are 111 colums and not all columns are visible
2. We can see that there are many missing values and we need to handle them
3. Not all rows might be required for our analysis and we might drop a few columns
4. There are some additional data like('%', 'months'...) added to the rows, we need to clean the data and also if required create derivative columns

### Lets look at all the columns and shape of the dataset before dropping rows/columns

In [ ]:
# shape of dataset
loan.shape

In [ ]:
#so we have total 39717 entries of loan and 111 coloumns defining the details of a particular loan 
# Lets check the info about the dataframe we will also print all the coloumns to check there dtypes 
#and see if we need to change any coloumns dtype
loan.info(verbose=True)

In [ ]:
loan['issue_d']

In [ ]:
# There are some columns with not so relevant column names which are useful in better understanding.
loan = loan.rename(columns = {'loan_amnt' : 'loan_amount', 'funded_amnt' : 'funded_amount', 'int_rate' : 'interest_rate', 'annual_inc' : 'annual_income', 'total_pymnt' : 'total_payment'})

<h2><center>Lets clean the dataset</center></h2>

### Remove additional values

In [ ]:
# remove 'months' from terms column, convert it to integer and rename the column to terms_in_months
loan['term'] = loan['term'].str.replace(' months', '').astype(int)

# rename the column
loan.rename(columns={'term': 'terms_in_months'}, inplace=True)

In [ ]:
# same goes for interest_rate column
loan['interest_rate'] = loan['interest_rate'].str.replace('%', '').astype(float)

## Identify and handle missing values

### Check and remove any duplicate Row

In [ ]:
loan.duplicated().sum()

Looks like there are no duplicates

### Check for missing values

In [ ]:
loan.isnull().sum()

### Observation

1. Some of the columns have all the null values and looks like these coluns might not be needed

In [ ]:
#calculate the percentage of null values in a columns and print the column have percentage more then 60 
# as null values can impact the data analysis
nullValuePercentage= round((loan.isnull().sum()/len(loan))*100,2)
nullValuePercentage[nullValuePercentage>60].sort_values(ascending=False)

1. From the above we see that there are quite a few columns which are completely null and these might not add any additional insights.
2. Lets drop these columns and also drop the columns and also drop columns where null value percentage is greater than 60

In [ ]:
#removing the columns having more than 60% of null value
loan = loan.loc[:,(loan.isnull().sum()/len(loan)*100)<60]

loan.shape

Looks like almost half of the columns have been dropped which contains null values greater than 60 percent

Lets further analyse the column and check their importance and decide if they are useful for our analysis else drop those columns

In [ ]:
# again check the percentage of null value in the coloumns 
print(round((loan.isnull().sum()/len(loan))*100,2).sort_values(ascending=False))

In [ ]:
# # There are few more columns which doesnt add any value like the last 3 digits of a zip_code, description and Title are very similar, id, member_id, url .... Lets drop these columns
# loan.drop( ['id', 'member_id', 'emp_title', 'collection_recovery_fee','url', 'desc',
#             'title','zip_code','inq_last_6mths','collection_recovery_fee',
#             'recoveries','tax_liens','total_rec_late_fee','policy_code','last_pymnt_d',
#             'earliest_cr_line','total_rec_int','total_rec_prncp','delinq_2yrs', 'open_acc', 'pub_rec', 'total_acc', 'last_pymnt_amnt','total_pymnt_inv','total_pymnt', 'chargeoff_within_12_mths','collections_12_mths_ex_med','revol_util','revol_bal','last_credit_pull_d'],axis=1,inplace=True)

In [ ]:
def analyze_outliers(dataframe, column_name):
    """
    Generates a boxplot for the specified column and displays statistical details.
    
    Parameters:
    - dataframe (pd.DataFrame): The DataFrame containing the data.
    - column_name (str): The name of the column to analyze.

    Returns:
    - None; outputs a boxplot and prints statistical details of the column.
    """
    if column_name in dataframe.columns:
        # Generate boxplot
        plt.figure(figsize=(10, 6))
        sns.boxplot(x=dataframe[column_name])
        plt.title(f'Boxplot for {column_name}')
        plt.grid(True)
        plt.show()

        # Calculate and display statistics
        stats = dataframe[column_name].describe()
        additional_stats = pd.Series({
            'mean': dataframe[column_name].mean(),
            'std_dev': dataframe[column_name].std(),
            'variance': dataframe[column_name].var()
        })
        
        # Using concat to combine the statistics
        all_stats = pd.concat([stats, additional_stats])
        print("\nStatistical Details:")
        print(all_stats)
    else:
        print("Column name provided does not exist in the dataframe.")

In [ ]:
analyze_outliers(loan, 'loan_amount')

In [ ]:
def analyze_outliers(dataframe, column_name):
    if column_name in dataframe.columns:
        # Create a boxplot to visualize outliers
        fig, ax = plt.subplots(figsize=(10, 6))
        sns.boxplot(x=dataframe[column_name], ax=ax)
        plt.grid(True)
        plt.title(f'Boxplot for {column_name}')
        plt.show()

        # Compute basic statistics
        stats = dataframe[column_name].describe()

        # Compute additional statistics
        additional_stats = pd.Series({
            'mean': dataframe[column_name].mean(),
            'std_dev': dataframe[column_name].std(),
            'variance': dataframe[column_name].var()
        })

        # Combine all statistics
        all_stats = pd.concat([stats, additional_stats])

        # Format all statistics for human-readable output
        readable_stats = {key: f"{value:,.2f}" for key, value in all_stats.items()}

        # Print the formatted statistics
        print("\nStatistical Details:")
        for key, value in readable_stats.items():
            print(f"{key.capitalize()}: {value}")

    else:
        print("Column name provided does not exist in the dataframe.")

In [ ]:
analyze_outliers(loan, 'loan_amount')

## Observations:

1. There are few points which lie outside of the IQR but these look like continous values and wide range.

In [ ]:
analyze_outliers(loan, 'annual_income')

## Observations:

1. From the above graph, you can see there are few points which are extreme right and far away from the IQR range.
2. To have meaningful insights, we will go ahead and drop the data points which are greater than 95 percentile as these points are considerd outliers.
3. Lets remove the outliers are then plot the box again

In [ ]:
# remove outliers and box plot again
loan = loan[loan['annual_income'] < loan['annual_income'].quantile(0.95)]

analyze_outliers(loan, 'annual_income')

In [ ]:
analyze_outliers(loan, 'installment')

In [ ]:
analyze_outliers(loan, 'interest_rate')

In [ ]:
# Binning the loan amounts into categories
bins = [0, 5000, 10000, 15000, 20000, 25000, 30000, 35000]
labels = ['0-5k', '5k-10k', '10k-15k', '15k-20k', '20k-25k', '25k-30k', '30k-35k']
loan['loan_amount_binned'] = pd.cut(loan['loan_amount'], bins=bins, labels=labels, include_lowest=True)

# Grouping by binned column and calculating mean interest rate
grouped = loan.groupby('loan_amount_binned')['interest_rate'].mean().reset_index()

# Show the plots
plt.figure(figsize=(10, 6))
sns.barplot(x='loan_amount_binned', y='interest_rate', data=grouped)
plt.title('Average Interest Rate by Loan Amount Category')
plt.xlabel('Loan Amount Category')
plt.ylabel('Average Interest Rate (%)')
plt.xticks(rotation=45)
plt.show()

## Observations:
1. The loan amout is categorized into bins of 5000 intervals.
2. We can see that the interest rate is directly proportional to the loan amount

In [ ]:
# Set up the matplotlib figure
plt.figure(figsize=(12, 6))  # Increase figure size for better visibility of both plots

# Create the first subplot for the boxplot
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot
sns.boxplot(x='grade', y='interest_rate', data=loan, order=sorted(loan['grade'].unique()))
plt.title('Interest Rate Distribution by Loan Grade')
plt.xlabel('Loan Grade')
plt.ylabel('Interest Rate (%)')

# Create the second subplot for the violin plot
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot
sns.violinplot(x='grade', y='interest_rate', data=loan, order=sorted(loan['grade'].unique()))
plt.title('Interest Rate Density and Distribution by Loan Grade')
plt.xlabel('Loan Grade')
plt.ylabel('Interest Rate (%)')

# Show the plots
plt.tight_layout()
plt.show()

## Observations:
1. There is a clear trend upward in interest rates as the loan grades progress from A to G
2. The lower grades have close IQR (A, B, C) than compared to other grades
3. The spread of violin plots in lower grades indicate that the variations are less and also there are more outliers present in higher grades

In [ ]:
import matplotlib.pyplot as plt

# Set up the matplotlib figure
plt.figure(figsize=(12, 5))

# First subplot for annual income
plt.subplot(1, 2, 1)  # 1 row, 2 columns, first subplot
plt.hist(loan['annual_income'], bins=50, color='blue', alpha=0.7)
plt.title('Distribution of Annual Income')
plt.xlabel('Annual Income ($)')
plt.ylabel('Frequency')

# Second subplot for loan amount
plt.subplot(1, 2, 2)  # 1 row, 2 columns, second subplot
plt.hist(loan['loan_amount'], bins=50, color='green', alpha=0.7)
plt.title('Distribution of Loan Amount')
plt.xlabel('Loan Amount ($)')
plt.ylabel('Frequency')

# Show the plots
plt.tight_layout()  # Adjusts subplot params so that subplots are nicely fit in the figure area.
plt.show()


## Observations:
1. There is no definite trend with the annual income and the frequency of loan amount.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))  # Adjust layout and size as needed
home_ownerships = loan['home_ownership'].unique()
for ax, home_ownership in zip(axes, home_ownerships):
    subset = loan[loan['home_ownership'] == home_ownership]
    counts = subset['loan_status'].value_counts()
    ax.pie(counts, labels=counts.index, autopct='%1.1f%%', startangle=90)
    ax.set_title(f'Loan Status for {home_ownership}')
plt.show()

## Observations
1. The majority of loans across all home ownerships are fully paid. Only a small portion of loans is still ongoing.
2. There is no significant difference in the percentages of fully paid, charged-off and current across all the status.

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='loan_status', y='dti', data=loan)
plt.title('DTI Ratios by Loan Status')
plt.xlabel('Loan Status')
plt.ylabel('Debt-to-Income Ratio (DTI)')
plt.xticks(rotation=45)
plt.show()

In [ ]:
import pandas as pd

# Encoding loan status as numeric
loan['status_numeric'] = loan['loan_status'].map({'Fully Paid': 0, 'Current': 0, 'Charged Off': 1, 'Default': 1})

# Calculating the correlation
correlation = loan[['dti', 'status_numeric']].corr().iloc[0, 1]
print(f"Correlation coefficient between DTI and Default Rate: {correlation:.3f}")


## Observations
1. The median is almost same for all the three types with slightly upward trend for current type.
2. The IQR is also same for the three types with `charged off` having slightly higher variance.
3. The coorelation coeffienct explain the weaker relation between DTI to loan status

In [ ]:
loan['emp_length'] = loan['emp_length'].astype(str)  # Convert all to string
loan['emp_length'] = loan['emp_length'].replace('nan', 'Not Specified')  # Replace NaN values if any

# Custom sort order, assuming 'emp_length' includes categories like '10+ years', '< 1 year', etc.
order = ['< 1 year', '1 year', '2 years', '3 years', '4 years', '5 years', '6 years', '7 years', '8 years', '9 years', '10+ years', 'Not Specified']

# Show the plots
plt.figure(figsize=(10, 6))
sns.countplot(x='emp_length', hue='loan_status', data=loan, order=order)
plt.title('Loan Status Distribution by Employment Length')
plt.xlabel('Employment Length')
plt.ylabel('Count of Loans')
plt.xticks(rotation=45)
plt.show()

## Observations:
1. Fully paid status is high across all the employment types.
2. The charged off staus doesnt significantly corelate with the employment length
3. There is relatively high current status in 10+ years of employment length

In [ ]:
top_titles = loan['emp_title'].value_counts().nlargest(10).index  # top 10 titles
filtered_data = loan[loan['emp_title'].isin(top_titles)]
avg_loan_amounts = filtered_data.groupby('emp_title')['loan_amount'].mean().sort_values()

# Show the plots
plt.figure(figsize=(10, 6))
sns.barplot(x=avg_loan_amounts.values, y=avg_loan_amounts.index)
plt.xlabel('Average Loan Amount ($)')
plt.ylabel('Employment Title')
plt.title('Average Loan Amount by Employment Title for Top 10 Titles')
plt.show()

In [ ]:
# Creating a cross-tabulation of loan status by top employment titles
status_distribution = pd.crosstab(filtered_data['emp_title'], filtered_data['loan_status'])

# Normalizing the data by rows to get proportions
status_distribution = status_distribution.div(status_distribution.sum(axis=1), axis=0)

# Show the plots
status_distribution.plot(kind='barh', stacked=True, figsize=(10, 8))
plt.title('Loan Status Distribution Across Top Employment Titles')
plt.xlabel('Proportion of Loan Statuses')
plt.ylabel('Employment Title')
plt.legend(title='Loan Status')
plt.show()

In [ ]:
avg_interest_rates = filtered_data.groupby('emp_title')['interest_rate'].mean().sort_values()

# Show the plots
plt.figure(figsize=(10, 6))
sns.barplot(x=avg_interest_rates.values, y=avg_interest_rates.index)
plt.xlabel('Average Interest Rate (%)')
plt.ylabel('Employment Title')
plt.title('Average Interest Rate by Employment Title for Top 10 Titles')
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(10, 10))

# First subplot: Average Loan Amounts
avg_loan_amounts = filtered_data.groupby('emp_title')['loan_amount'].mean().sort_values()
sns.barplot(x=avg_loan_amounts.values, y=avg_loan_amounts.index, ax=axes[0])
axes[0].set_title('Average Loan Amount by Employment Title for Top 10 Titles')
axes[0].set_xlabel('Average Loan Amount ($)')
axes[0].set_ylabel('Employment Title')

# Second subplot: Loan Status Distribution
status_distribution = pd.crosstab(filtered_data['emp_title'], filtered_data['loan_status'])
status_distribution = status_distribution.div(status_distribution.sum(axis=1), axis=0)
status_distribution.plot(kind='barh', stacked=True, ax=axes[1])
axes[1].set_title('Loan Status Distribution Across Top Employment Titles')
axes[1].set_xlabel('Proportion of Loan Statuses')
axes[1].set_ylabel('Employment Title')

# Third subplot: Average Interest Rates
avg_interest_rates = filtered_data.groupby('emp_title')['interest_rate'].mean().sort_values()
sns.barplot(x=avg_interest_rates.values, y=avg_interest_rates.index, ax=axes[2])
axes[2].set_title('Average Interest Rate by Employment Title for Top 10 Titles')
axes[2].set_xlabel('Average Interest Rate (%)')
axes[2].set_ylabel('Employment Title')

# Show the plots
plt.tight_layout()
plt.show()


## Observation:
1. Certain orgs like UPS, USAF appear to higher average loan amounts than other orgs
2. Few orgs like Walmart, UPS, Bank of America and At&t show a noticable amount of loans charged off. This might indicate that private companies tend to charge off their employee loans.
3. The interest rates show significant difference across the orgs altought US Army has higesht interest rate

In [ ]:
# Group by state and calculate average loan amount and interest rate
state_loan_data = loan.groupby('addr_state').agg({
    'loan_amount': 'mean',
    'interest_rate': 'mean'
}).reset_index()

# Show the plots
fig, ax1 = plt.subplots(figsize=(15, 7))
sns.barplot(x='addr_state', y='loan_amount', data=state_loan_data, ax=ax1, color='blue', alpha=0.6)
ax1.set_ylabel('Average Loan Amount ($)', color='blue')
ax2 = ax1.twinx()
sns.lineplot(x='addr_state', y='interest_rate', data=state_loan_data, ax=ax2, color='red', marker='o')
ax2.set_ylabel('Average Interest Rate (%)', color='red')
ax1.set_xlabel('State')
plt.title('Average Loan Amount and Interest Rate by State')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Define default as Charged Off or Default loan status
loan['is_default'] = loan['loan_status'].apply(lambda x: 1 if x in ['Charged Off', 'Default'] else 0)

# Group by state and calculate default rate
default_rates = loan.groupby('addr_state')['is_default'].mean().reset_index()

# Show the plots
plt.figure(figsize=(15, 7))
sns.barplot(x='addr_state', y='is_default', data=default_rates)
plt.ylabel('Default Rate')
plt.xlabel('State')
plt.title('Loan Default Rates by State')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Categorizing loans
loan['loan_outcome'] = loan['loan_status'].apply(lambda x: 'Good' if x in ['Fully Paid', 'Current'] else 'Bad')

# Count the occurrences of each loan outcome
loan_outcome_counts = loan['loan_outcome'].value_counts()

# Show the plots
plt.figure(figsize=(8, 8))
plt.pie(loan_outcome_counts, labels=loan_outcome_counts.index, autopct='%1.1f%%', startangle=90, colors=['#66b3ff', '#ff9999'])
plt.title('Distribution of Loan Outcomes')
plt.show()

In [ ]:
loan['issue_year'] = pd.to_datetime(loan['issue_d'], format='%b-%y', errors='coerce').dt.year
loans_per_year = loan.groupby('issue_year').size()
avg_loan_amount_per_year = loan.groupby('issue_year')['loan_amount'].mean()

# Setup the figure and axes
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(18, 6))  # Adjust size as needed for side by side presentation

# First plot: Number of Loans Issued Per Year
axes[0].bar(loans_per_year.index, loans_per_year.values, color='skyblue')
axes[0].set_title('Number of Loans Issued Per Year', fontsize=16)
axes[0].set_xlabel('Year', fontsize=14)
axes[0].set_ylabel('Number of Loans', fontsize=14)

# Second plot: Average Loan Amount Issued Per Year
sns.barplot(x='issue_year', y='loan_amount', data=loan, ax=axes[1])
axes[1].set_title('Average Loan Amount Issued Per Year', fontsize=16)
axes[1].set_xlabel('Year', fontsize=14)
axes[1].set_ylabel('Average Loan Amount ($)', fontsize=14)

# Show the plots
plt.tight_layout()
plt.show()